In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from scipy import stats
import os
import cv2
from PIL import Image
from moviepy.editor import VideoFileClip
from IPython.display import HTML

***Image Processing Functions***

In [ ]:
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
       
def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    if lines is None:
        return
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img, lines

def weighted_img(img, initial_img, a=0.8, b=1., g=0.):
    return cv2.addWeighted(initial_img,a, img, b, g)

In [ ]:
def change_color_space(img, color_space='HSV'):
    if color_space == 'HSV':
        space = cv2.COLOR_RGB2HSV
    if color_space == 'HLS':
        space = cv2.COLOR_RGB2HLS
    if color_space == 'LAB':
        space = cv2.COLOR_RGB2LAB
    else:
        space = None
    if space is not None:
        return cv2.cvtColor(img, space)    
    return img

In [ ]:
def extract_line(img, lower, upper):
    mask = cv2.inRange(img, np.array(lower, dtype=np.uint8), np.array(upper, dtype=np.uint8))
    return mask

***Region of Interest***

In [ ]:
def get_vertices_for_img(img):
    height, width = img.shape[0], img.shape[1]
    if (width, height) == (960, 540):
        region_bottom_left = (130 ,height - 1)
        region_top_left = (410, 330)
        region_top_right = (650, 350)
        region_bottom_right = (width - 30, height - 1)
    else:
        region_bottom_left = (200 , 680)
        region_top_left = (600, 450)
        region_top_right = (750, 450)
        region_bottom_right = (1100, 650)
    return np.array([[region_bottom_left, region_top_left, region_top_right, region_bottom_right]], dtype=np.int32)

In [ ]:
def region_of_interest(img):
    mask = np.zeros_like(img)
    if len(img.shape) > 2:
        ignore_mask_color = (255,) * img.shape[2]
    else:
        ignore_mask_color = 255
    vert = get_vertices_for_img(img)
    cv2.fillPoly(mask, vert, ignore_mask_color)
    return cv2.bitwise_and(img, mask)

***Lane line detection functions***

In [ ]:
def find_lane_lines_formula(lines):
    xs, ys = [], []
    for line in lines:
        for x1, y1, x2, y2 in line:
            xs.extend([x1, x2])
            ys.extend([y1, y2])
    if len(xs) == 0:
        return 0, 0
    slope, intercept, _, _, _ = stats.linregress(xs, ys)
    return slope, intercept

In [ ]:
def draw_full_lines(img, slope, intercept, color, thickness):
    if slope == 0 and intercept == 0:
        return
    y = np.array([int(img.shape[0]*0.63), img.shape[0]-1], 'float')
    x = (y - intercept) / slope
    cv2.line(img, (int(x[0]), int(y[0])), (int(x[1]), int(y[1])), color, thickness)

***Process single image***

In [ ]:
def process_image(image):
    # Convert color spaces
    LAB_img = change_color_space(image,'LAB')
    HLS_img = change_color_space(image,'HLS')

    # Extract yellow and white lines
    yellow_lines = extract_line(LAB_img, [100,100,150], [220,180,255])
    white_lines = extract_line(HLS_img, [0,200,0], [180,255,255])

    # Morphological operations for white lines
    white_lines = cv2.dilate(white_lines, np.ones((5,5), np.uint8), iterations=2)
    white_lines = cv2.erode(white_lines, np.ones((5,5), np.uint8), iterations=2)
    white_lines = cv2.dilate(white_lines, np.ones((5,5), np.uint8), iterations=1)
    
    # Combine masks and apply to image
    line_mask = yellow_lines + white_lines
    masked_img = np.copy(image)
    masked_img = cv2.dilate(masked_img, np.ones((5,5), np.uint8), iterations=2)
    masked_img = cv2.erode(masked_img, np.ones((5,5), np.uint8), iterations=2)
    masked_img = cv2.dilate(masked_img, np.ones((5,5), np.uint8), iterations=1)
    masked_img[line_mask!=255] = [0,0,0]
    
    # Region of interest and Hough Transform
    cleaned_img = region_of_interest(masked_img)
    hough_img, lines = hough_lines(grayscale(cleaned_img), 1, np.pi/180, 17, 7, 0)
    
    # Separate left/right lanes
    overlay = np.zeros_like(image)
    right_lanes, left_lanes = [], []
    epsilon = 0.5
    middle_x = image.shape[1]/2
    
    for line in lines if lines is not None else []:
        x1, y1, x2, y2 = line[0]
        if (x2-x1)!=0 and (y2-y1)!=0:
            slope = (y2-y1)/(x2-x1)
            if abs(slope) > epsilon:
                if slope > 0 and middle_x < x1 < x2:
                    right_lanes.append([[x1,y1,x2,y2]])
                elif slope < 0 and x1 < x2 < middle_x:
                    left_lanes.append([[x1,y1,x2,y2]])

    # Draw full lanes   
    if len(right_lanes) > 0:
        slope, intercept = find_lane_lines_formula(right_lanes)
        draw_full_lines(overlay, slope, intercept, [0,0,255], 10)
    if len(left_lanes) > 0:
        slope, intercept = find_lane_lines_formula(left_lanes)
        draw_full_lines(overlay, slope, intercept, [0,0,255], 10)
    
    # Overlay on original image
    result = weighted_img(overlay, image, a=0.8, b=1., g=0.)
    return result

In [ ]:
image_folder = "Test_Images/"
output_folder = "Test_Images_Output"
os.makedirs(output_folder, exist_ok=True)

***Process images***

In [ ]:
for name_img in os.listdir(image_folder):
    image_path = os.path.join(image_folder, name_img)
    image = mpimg.imread(image_path)
    
    result, masked_img, hough_img, yellow_lines, white_lines = process_image(image)
    
    # Show step-by-step visualizations
    plt.figure(figsize=(15,5))
    plt.subplot(1,3,1)
    plt.imshow(image)
    plt.title('Original RGB')
    plt.subplot(1,3,2)
    plt.imshow(masked_img)
    plt.title('Masked Image')
    plt.subplot(1,3,3)
    plt.imshow(result)
    plt.title('Final Lane Overlay')
    plt.show()
    
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.imshow(yellow_lines, cmap='gray')
    plt.title('Yellow Mask')
    plt.subplot(1,2,2)
    plt.imshow(white_lines, cmap='gray')
    plt.title('White Mask')
    plt.show()
    
    plt.figure(figsize=(8,6))
    plt.imshow(hough_img, cmap='gray')
    plt.title('Hough Lines Reconstruction')
    plt.show()
    
    # Save final overlay
    Image.fromarray(result).save(os.path.join(output_folder, name_img))

***Process video***

In [ ]:
video_input_folder = "Test_Videos/"
video_output_folder = "Test_Videos_Output"
os.makedirs(video_output_folder, exist_ok=True)

In [ ]:
for video_name in os.listdir(video_input_folder):
    input_path = os.path.join(video_input_folder, video_name)
    output_path = os.path.join(video_output_folder, video_name)
    clip = VideoFileClip(input_path)
    processed_clip = clip.fl_image(lambda frame: process_image(frame)[0])
    processed_clip.write_videofile(output_path, audio=False)
    print(f"Processed video saved at: {output_path}")